In [1]:
!pip install pennylane --quiet
!pip install pennylane-lightning --quiet
!pip install autograd --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 37.5 MB/s eta 0:00:00


In [2]:
import pennylane as qml
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane import numpy as np
from time import time

np.random.seed(0)

In [52]:
num_qubits = 16
num_layers = 6
weights_init = np.random.uniform(-np.pi, np.pi, (num_layers, num_qubits, 1))
bias_init = np.array([0.])

In [4]:
import tensorflow as tf
import numpy as onp
import collections

"""
Module for MNIST Digits Dataset preprocessing.
https://www.tensorflow.org/quantum/tutorials/mnist

Python 3.10.11
"""

def filter_by_classes(x, y, classes=[3,6]):
    """
    Function that filters the MNIST Digits Dataset and returns samples on 'classes'.
    Parameters:
        x: Sample images.
        y: Sample labels.
        classes: List of classes to filter.
    Returns:
        x: x filtered by 'classes'.
        y: x filtered by 'classes'.
    """
    if not all(np.isin(classes, range(0, 10))):
        return ValueError("Classes must be a list of digits (0-9).")
    x, y = x[np.isin(y, classes)], y[np.isin(y, classes)]
    if len(classes)==2:
        return x, y==classes[-1]
    else:
        return x, y

def remove_contradicting(xs, ys):

    mapping = collections.defaultdict(set)
    orig_x = {}
    # Determine the set of labels for each unique image:
    for x,y in zip(xs,ys):
       orig_x[tuple(x.flatten())] = x
       mapping[tuple(x.flatten())].add(y)

    new_x = []
    new_y = []
    for flatten_x in mapping:
      x = orig_x[flatten_x]
      labels = mapping[flatten_x]
      if len(labels) == 1:
          new_x.append(x)
          new_y.append(next(iter(labels)))
      else:
          # Throw out images that match more than one label.
          pass

    num_uniq_3 = sum(1 for value in mapping.values() if len(value) == 1 and True in value)
    num_uniq_6 = sum(1 for value in mapping.values() if len(value) == 1 and False in value)
    num_uniq_both = sum(1 for value in mapping.values() if len(value) == 2)

    print("Number of unique images:", len(mapping.values()))
    print("Number of unique 3s: ", num_uniq_3)
    print("Number of unique 6s: ", num_uniq_6)
    print("Number of unique contradicting labels (both 3 and 6): ", num_uniq_both)
    print()
    print("Initial number of images: ", len(xs))
    print("Remaining non-contradicting unique images: ", len(new_x))

    return np.array(new_x), np.array(new_y)

def preprocess_mnist_digits(classes=[3,6]):
    """"
    Function that downloads the MNIST Digits dataset with TensorFlow and performs the following tasks:
        1. Normalizes pixel values from (0, 255) to (0, 1).
        2. By default, returns only 2 classes of digits for classification (this can be deactivated or modified by the 'classes' parameter).
        3. Resizes samples to 4x4 images.
        4. Removes samples that belong to multiple classes simultaneously.
        5. Converts images to binary."
    Parameters:
    Returns:
    """

    # Download dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Rescale the images from [0,255] to the [0.0,1.0] range.
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

    # Filter to get only '3's and '6's
    x_train, y_train = filter_by_classes(x_train, y_train, classes=classes)
    x_test, y_test = filter_by_classes(x_test, y_test, classes=classes)

    print("Number of filtered training examples:", len(x_train))
    print("Number of filtered test examples:", len(x_test))

    # Resize images to 4x4
    x_train_small = tf.image.resize(x_train, (4,4)).numpy()
    x_test_small = tf.image.resize(x_test, (4,4)).numpy()

    x_train_nocon, y_train_nocon = remove_contradicting(x_train_small, y_train)

    THRESHOLD = 0.5

    # Converts non contradicting samples to binary via threshold and converting bool to float.
    x_train_bin = np.array(x_train_nocon > THRESHOLD, dtype=np.float32)
    x_test_bin = np.array(x_test_small > THRESHOLD, dtype=np.float32)

    return x_train_bin.reshape(-1, 16), y_train_nocon, x_test_bin.reshape(-1, 16), y_test



In [5]:
X_train, Y_train, X_test, Y_test = preprocess_mnist_digits()
Y_train = np.where(Y_train==False, -1, 1)
Y_test = np.where(Y_test==False, -1, 1)

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
Number of filtered training examples: 12049
Number of filtered test examples: 1968
Number of unique images: 10387
Number of unique 3s:  5426
Number of unique 6s:  4912
Number of unique contradicting labels (both 3 and 6):  49

Initial number of images:  12049
Remaining non-contradicting unique images:  10338
(10338, 16) (10338,) (1968, 16) (1968,)


In [99]:
dev = qml.device("lightning.qubit", wires=range(-1, 16))

@qml.qnode(dev)
def circuit(params, data):

    qml.PauliX(wires=dev.wires[0])
    qml.BasisState(data, wires=dev.wires[1:])

    for i in range(params.shape[0]):
        for j in range(1, 16):
            # print(i, j)
            if i%2 == 0:
                qml.PauliRot(params[i, j], "ZX", wires=[dev.wires[j], dev.wires[0]])
            else:
                qml.PauliRot(params[i, j], "XX", wires=[dev.wires[j], dev.wires[0]])


    # qml.Hadamard(wires=dev.wires[0])

    ## measure in Y basis

    # qml.adjoint(qml.S(wires=dev.wires[0])) ## y basis
    # qml.Hadamard(wires=dev.wires[0]) ## y basis

    #print(qml.expval(qml.PauliY(wires=dev.wires[0])))
    return qml.expval(qml.PauliY(wires=dev.wires[0]))
    # return qml.probs(wires = dev.wires[0])


In [100]:
for i in range(100):
  print(circuit(weights_init, X_train[i]))

0.00013296389304577548
-0.0002277243013287532
0.0010084006194485834
0.0003929851159948609
-0.0007266871347283105
-0.0006664773523226861
0.0002554935243422122
-0.0007266871347283105
-0.00014720683808501023
0.001041434965938345
0.0007374956946047663
-0.0006664773523226861
0.001041434965938345
-0.0007266871347283105
0.001041434965938345
0.00013296389304577548
0.00102929993394569
0.00032854730206595515
-0.0008939150310021594
-0.0007266871347283105
0.00013296389304577548
0.0009814264166278128
-0.0002277243013287532
0.0010084006194485834
-0.0008939150310021594
-0.00014720683808501023
-0.0002277243013287532
-0.0002277243013287532
-0.0008217799135693882
-0.0006497143679339889
-0.0007110372457249528
-0.0007266871347283105
-0.0007266871347283105
-0.0002277243013287532
-0.0007266871347283105
-0.0008217799135693882


KeyboardInterrupt: ignored

In [58]:
weights_init

tensor([[[ 1.29794716],
         [-0.53497007],
         [-0.87621809],
         [ 2.0650123 ],
         [ 2.67014586],
         [-2.85252019],
         [-1.67995415],
         [-0.95178087],
         [ 1.97899276],
         [ 3.0504326 ],
         [ 2.94663612],
         [ 2.54436549],
         [-1.27827469],
         [ 3.09139782],
         [-1.57444032],
         [-2.47616466]],

        [[ 2.83341882],
         [-1.67496993],
         [ 1.19234917],
         [-2.77492884],
         [ 1.44958802],
         [ 2.39841883],
         [-1.42982115],
         [-0.75990793],
         [-0.78982037],
         [ 1.56318272],
         [-1.64740568],
         [-2.06180779],
         [-0.31860997],
         [-1.22856123],
         [ 2.13118811],
         [-1.64781671]],

        [[ 0.0150134 ],
         [ 2.78083477],
         [ 0.84193237],
         [ 2.3077474 ],
         [ 2.76591905],
         [ 1.5756021 ],
         [ 1.25396709],
         [ 2.94031437],
         [ 3.10641178],
         [-0

In [ ]:
X_train[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       dtype=float32, requires_grad=True)

In [102]:
def variational_classifier(weights, x):
    #return circuit(weights, x) + bias
    return circuit(weights, x)
    #return np.dot(circuit(weights, x), np.array([1, -1]))

In [103]:
# def square_loss(labels, predictions):
#     loss = 0
#     for l, p in zip(labels, predictions):
#         loss = loss + (l - p) ** 2

#     loss = loss / len(labels)
#     return loss

def loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss += 1 - l*p

    return loss/labels.shape[0]

# def loss(y_true, y_pred):
#     print(y_pred)
#     # print(y_true, y_pred)
#     # loss = 0
#     # for l, p in zip(labels, predictions):
#     #     loss += 1 - l*p

#     y_pred = np.array(y_pred).reshape(-1, 1)
#     #y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
#     loss = 1-y_true * y_pred

#     print(loss)

#     return loss/len(loss)


In [89]:
# def accuracy(labels, predictions):

#     loss = 0
#     for l, p in zip(labels, predictions):
#         if abs(l - p) < 1e-5:
#             loss = loss + 1
#     loss = loss / len(labels)

#     return loss

In [90]:
def cost(weights, X, Y):
    predictions = [variational_classifier(weights, x) for x in X]
    #return square_loss(Y, predictions)
    # print(loss(Y, predictions))
    return loss(Y, predictions)


In [91]:
opt = qml.AdamOptimizer(0.1)
batch_size = 10

In [ ]:
weights = weights_init
bias = bias_init

In [107]:
# 4 feaatures

# Experiment method 5

opt = qml.AdamOptimizer(stepsize = 0.001) # original 0.001
batch_size = 5
num_train = len(X_train)


# train the variational classifier
var_weights = weights_init
exp_time = time()
for it in range(0, 12001):

    # Gradient descent step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    feats_train_batch = X_train[batch_index]
    Y_train_batch = Y_train[batch_index]
    var_weights = opt.step(lambda v_weights: cost(v_weights, feats_train_batch, Y_train_batch), var_weights)


    if it % 200 == 0:
      print(it)
      print("cost:", cost(var_weights, feats_train_batch, Y_train_batch))
      print(repr(var_weights))
      print(f"--------time VarQMC Epoch: {it}---------------")
      print(time() - exp_time)
      exp_time = time()

    if it % 800 == 0:
      print(it)
      print("cost:", cost(var_weights, X_train[:100], Y_train[:100]))
      print(repr(var_weights))

0
cost: 1.0000347855618872
tensor([[[ 1.29794716],
         [-0.53397039],
         [-0.87521841],
         [ 2.06601127],
         [ 2.67114554],
         [-2.85152052],
         [-1.6809518 ],
         [-0.95277942],
         [ 1.97999243],
         [ 3.05143228],
         [ 2.94563637],
         [ 2.54336585],
         [-1.27727501],
         [ 3.09239749],
         [-1.57344056],
         [-2.47516498]],

        [[ 2.83341882],
         [-1.67596907],
         [ 1.191351  ],
         [-2.77592841],
         [ 1.45058667],
         [ 2.39941814],
         [-1.43082047],
         [-0.75890947],
         [-0.79081992],
         [ 1.56418196],
         [-1.64840456],
         [-2.06081005],
         [-0.31761023],
         [-1.22956015],
         [ 2.13018816],
         [-1.64881588]],

        [[ 0.0150134 ],
         [ 2.7798352 ],
         [ 0.84293165],
         [ 2.30874598],
         [ 2.76691793],
         [ 1.57460264],
         [ 1.25496618],
         [ 2.93931833],
         

KeyboardInterrupt: ignored

In [ ]:
np.save("arr.npy", weights)

In [ ]:
accuracy(Y_test, [np.sign(variational_classifier(weights, bias, x)) for x in X_test])

KeyboardInterrupt: ignored

In [ ]:
[variational_classifier(weights, bias, x) for x in X_test[:20]]

[tensor([6.24689778], requires_grad=True),
 tensor([6.24701248], requires_grad=True),
 tensor([6.24701406], requires_grad=True),
 tensor([6.24700538], requires_grad=True),
 tensor([6.24704156], requires_grad=True),
 tensor([6.24704156], requires_grad=True),
 tensor([6.24695561], requires_grad=True),
 tensor([6.24696978], requires_grad=True),
 tensor([6.24689778], requires_grad=True),
 tensor([6.24701177], requires_grad=True),
 tensor([6.24706099], requires_grad=True),
 tensor([6.24689778], requires_grad=True),
 tensor([6.24700897], requires_grad=True),
 tensor([6.24706099], requires_grad=True),
 tensor([6.24692834], requires_grad=True),
 tensor([6.24706099], requires_grad=True),
 tensor([6.24701406], requires_grad=True),
 tensor([6.24703736], requires_grad=True),
 tensor([6.24692834], requires_grad=True),
 tensor([6.24691144], requires_grad=True)]

In [ ]:
np.unique(Y_test[:20], return_counts=True)

(array([-1,  1]), array([11,  9]))